In [1]:
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 读取点云数据和标签
def load_labeled_point_cloud(file_path):
    data = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 4:
                x, y, z, label = map(float, parts)
                data.append([x, y, z])
                labels.append(int(label))
    return np.array(data), np.array(labels)

def create_voxel_grid(data, labels, grid_size_x, grid_size_y, grid_size_z):
    if data.size == 0 or labels.size == 0:
        print(f"Warning: Empty data or labels array. data.size={data.size}, labels.size={labels.size}")
        return None, None

    grid = np.zeros((grid_size_x, grid_size_y, grid_size_z))
    label_grid = np.zeros((grid_size_x, grid_size_y, grid_size_z))

    min_coords = np.min(data, axis=0)
    max_coords = np.max(data, axis=0)
    voxel_dim = (max_coords - min_coords) / [grid_size_x, grid_size_y, grid_size_z]

    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, [0, 0, 0], [grid_size_x-1, grid_size_y-1, grid_size_z-1])  # Ensure indices are within bounds
        grid[voxel[0], voxel[1], voxel[2]] = 1
        label_grid[voxel[0], voxel[1], voxel[2]] = labels[i] - 1  # Convert labels 1 and 2 to 0 and 1

    return grid, label_grid

def load_data_from_directory(data_dir, grid_size_x=32, grid_size_y=32, grid_size_z=64):
    x_data = []
    y_data = []
    for file in os.listdir(data_dir):
        if file.endswith("_labeled.txt"):
            file_path = os.path.join(data_dir, file)
            data, labels = load_labeled_point_cloud(file_path)
            if data.size == 0 or labels.size == 0:
                print(f"Skipping empty file: {file_path}")
                continue
            voxel_grid, label_grid = create_voxel_grid(data, labels, grid_size_x, grid_size_y, grid_size_z)
            if voxel_grid is not None and label_grid is not None:
                # 应用数据增强
                voxel_grid = augment_voxel_grid(voxel_grid)
                x_data.append(voxel_grid)
                y_data.append(label_grid)
    x_data = np.expand_dims(np.array(x_data), axis=-1)
    y_data = np.expand_dims(np.array(y_data), axis=-1)
    return x_data, y_data

# 数据增强
def augment_voxel_grid(voxel_grid):
    """
    对体素网格进行随机旋转和翻转。
    """
    # 随机旋转
    angle = np.random.uniform(0, 360)
    voxel_grid = np.rot90(voxel_grid, k=int(angle // 90), axes=(0, 1))
    
    # 随机翻转
    if np.random.rand() > 0.5:
        voxel_grid = np.flip(voxel_grid, axis=0)
    if np.random.rand() > 0.5:
        voxel_grid = np.flip(voxel_grid, axis=1)
    if np.random.rand() > 0.5:
        voxel_grid = np.flip(voxel_grid, axis=2)
    
    return voxel_grid

# 定义UNET模型
# 定义UNET模型
def unet_3d(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Encoder
    conv1 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(inputs)
    conv1 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv1)
    pool1 = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(conv1)

    conv2 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(pool1)
    conv2 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv2)
    pool2 = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(pool2)
    conv3 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv3)
    pool3 = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(conv3)

    # Bottleneck
    conv4 = tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(pool3)
    conv4 = tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv4)

    # Decoder
    up5 = tf.keras.layers.Conv3DTranspose(256, 2, strides=(2, 2, 2), padding='same')(conv4)
    concat5 = tf.keras.layers.concatenate([up5, conv3], axis=-1)
    conv5 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(concat5)
    conv5 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv5)

    up6 = tf.keras.layers.Conv3DTranspose(128, 2, strides=(2, 2, 2), padding='same')(conv5)
    concat6 = tf.keras.layers.concatenate([up6, conv2], axis=-1)
    conv6 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(concat6)
    conv6 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv6)

    up7 = tf.keras.layers.Conv3DTranspose(64, 2, strides=(2, 2, 2), padding='same')(conv6)
    concat7 = tf.keras.layers.concatenate([up7, conv1], axis=-1)
    conv7 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(concat7)
    conv7 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv7)

    outputs = tf.keras.layers.Conv3D(1, 1, activation='sigmoid')(conv7)  # 1 channel output for binary classification

    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    return model

# 定义学习率调度函数
def scheduler(epoch, lr):
    if epoch < 10:
        return float(lr)
    else:
        return float(lr * tf.math.exp(-0.2).numpy())

# 加载数据
data_dir = 'D:\BaiduSyncdisk\新建文件夹\桌面文件\深度学习程序\A-轮廓线识别\预处理后的数据1124个\预处理后的数据'
x_data, y_data = load_data_from_directory(data_dir, grid_size_x=32, grid_size_y=32, grid_size_z=64)

# 确保数据归一化
x_data = x_data / np.max(x_data)
y_data = y_data / np.max(y_data)

# 划分数据集
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42
)

# 定义模型
input_shape = (32, 32, 64, 1)  # 32x32x64 体素尺寸
model = unet_3d(input_shape)

# 编译模型
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00003), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# 定义回调函数
callbacks = [
    tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('unet3d_best_model.keras', save_best_only=True, verbose=1)  # 改为 .keras 格式
]

# 训练模型
history = model.fit(
    x_train, y_train, 
    validation_split=0.1,
    epochs=20, 
    batch_size=10,
    callbacks=callbacks
)

# 评估模型
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# 绘制训练过程中的损失和精确度
plot_training_history(history)


<>:130: SyntaxWarning: invalid escape sequence '\B'
<>:130: SyntaxWarning: invalid escape sequence '\B'
C:\Users\51625\AppData\Local\Temp\ipykernel_29236\4114662300.py:130: SyntaxWarning: invalid escape sequence '\B'
  data_dir = 'D:\BaiduSyncdisk\新建文件夹\桌面文件\深度学习程序\A-轮廓线识别\预处理后的数据1124个\预处理后的数据'



Epoch 1: LearningRateScheduler setting learning rate to 2.9999999242136255e-05.
Epoch 1/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9589 - loss: 0.7615
Epoch 1: val_loss improved from inf to 0.35829, saving model to unet3d_best_model.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 596s 7s/step - accuracy: 0.9591 - loss: 0.7591 - val_accuracy: 0.9782 - val_loss: 0.3583 - learning_rate: 3.0000e-05

Epoch 2: LearningRateScheduler setting learning rate to 2.9999999242136255e-05.
Epoch 2/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9773 - loss: 0.3292
Epoch 2: val_loss improved from 0.35829 to 0.27074, saving model to unet3d_best_model.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 647s 8s/step - accuracy: 0.9773 - loss: 0.3289 - val_accuracy: 0.9782 - val_loss: 0.2707 - learning_rate: 3.0000e-05

Epoch 3: LearningRateScheduler setting learning rate to 2.9999999242136255e-05.
Epoch 3/20
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9769 - loss: 0.2672
Epoch 3: val_loss improved from 0.27074 

C:\Users\51625\AppData\Local\Temp\ipykernel_29236\4114662300.py:130: SyntaxWarning: invalid escape sequence '\B'
  data_dir = 'D:\BaiduSyncdisk\新建文件夹\桌面文件\深度学习程序\A-轮廓线识别\预处理后的数据1124个\预处理后的数据'


NameError: name 'plot_training_history' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

# 加载模型
model = load_model('D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/模型-0725/unet_3d_model_0726.h5')

# 加载测试数据
# 假设你已经有预处理好的测试数据集
(x_test, y_test) = # 请在此处添加你的测试数据集加载代码

# 评估模型
loss, accuracy = model.evaluate(x_test, y_test)
print(f'模型在测试集上的损失：{loss}')
print(f'模型在测试集上的精度：{accuracy}')

# 加载训练过程中的损失和精确度数据
history = np.load('training_history.npy', allow_pickle=True).item()

# 绘制训练过程中的损失和精确度
plt.figure(figsize=(12, 5))

# 绘制损失
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='训练损失')
plt.plot(history['val_loss'], label='验证损失')
plt.title('训练和验证损失')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 绘制精确度
plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='训练精度')
plt.plot(history['val_accuracy'], label='验证精度')
plt.title('训练和验证精度')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [6]:
import tensorflow as tf

# 加载模型
model_load_path = 'D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/模型-0725/unet_3d_model_0726.h5'
model = tf.keras.models.load_model(model_load_path)
print(f"Model loaded from {model_load_path}")


import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# 读取点云数据和标签
def load_labeled_point_cloud(file_path):
    data = []
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 4:
                x, y, z, label = map(float, parts)
                data.append([x, y, z])
                labels.append(int(label))
    return np.array(data), np.array(labels)

def create_voxel_grid(data, labels, grid_size_x, grid_size_y, grid_size_z):
    if data.size == 0 or labels.size == 0:
        print(f"Warning: Empty data or labels array. data.size={data.size}, labels.size={labels.size}")
        return None, None

    grid = np.zeros((grid_size_x, grid_size_y, grid_size_z))
    label_grid = np.zeros((grid_size_x, grid_size_y, grid_size_z))

    min_coords = np.min(data, axis=0)
    max_coords = np.max(data, axis=0)
    voxel_dim = (max_coords - min_coords) / [grid_size_x, grid_size_y, grid_size_z]

    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, [0, 0, 0], [grid_size_x-1, grid_size_y-1, grid_size_z-1])  # Ensure indices are within bounds
        grid[voxel[0], voxel[1], voxel[2]] = 1
        label_grid[voxel[0], voxel[1], voxel[2]] = labels[i] - 1  # Convert labels 1 and 2 to 0 and 1

    return grid, label_grid


def load_data_from_directory(data_dir, grid_size_x=32, grid_size_y=32, grid_size_z=64):
    x_data = []
    y_data = []
    for file in os.listdir(data_dir):
        if file.endswith("_labeled.txt"):
            file_path = os.path.join(data_dir, file)
            data, labels = load_labeled_point_cloud(file_path)
            if data.size == 0 or labels.size == 0:
                print(f"Skipping empty file: {file_path}")
                continue
            voxel_grid, label_grid = create_voxel_grid(data, labels, grid_size_x, grid_size_y, grid_size_z)
            if voxel_grid is not None and label_grid is not None:
                # 应用数据增强
                voxel_grid = augment_voxel_grid(voxel_grid)
                x_data.append(voxel_grid)
                y_data.append(label_grid)
    x_data = np.expand_dims(np.array(x_data), axis=-1)
    y_data = np.expand_dims(np.array(y_data), axis=-1)
    return x_data, y_data


# 加载数据
data_dir = 'D:\BaiduSyncdisk\新建文件夹\桌面文件\深度学习程序\A-轮廓线识别\预处理后的数据1124个\预处理后的数据'
x_data, y_data = load_data_from_directory(data_dir, grid_size=16)

# 确保数据归一化
x_data = x_data / np.max(x_data)
y_data = y_data / np.max(y_data)

# 划分数据集
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2, random_state=42
)


# 评估模型
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# 绘制训练过程中的准确率和损失变化曲线
plt.figure(figsize=(12, 5))

# 绘制损失
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 绘制准确率
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

<>:71: SyntaxWarning: invalid escape sequence '\B'
<>:71: SyntaxWarning: invalid escape sequence '\B'
C:\Users\51625\AppData\Local\Temp\ipykernel_28316\945459812.py:71: SyntaxWarning: invalid escape sequence '\B'
  data_dir = 'D:\BaiduSyncdisk\新建文件夹\桌面文件\深度学习程序\A-轮廓线识别\预处理后的数据1124个\预处理后的数据'


Model loaded from D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/模型-0725/unet_3d_model_0726.h5


C:\Users\51625\AppData\Local\Temp\ipykernel_28316\945459812.py:71: SyntaxWarning: invalid escape sequence '\B'
  data_dir = 'D:\BaiduSyncdisk\新建文件夹\桌面文件\深度学习程序\A-轮廓线识别\预处理后的数据1124个\预处理后的数据'


TypeError: load_data_from_directory() got an unexpected keyword argument 'grid_size'

In [12]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

# 加载模型
model = load_model('D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/模型-0725/unet_3d_model_0726.h5')

# 加载训练过程中的损失和精确度数据
# 假设你在训练过程中使用了 model.fit 时将 history 对象保存为一个字典，并保存为 .npy 文件
history = np.load('plot_training_history.npy', allow_pickle=True).item()

# 绘制训练过程中的损失和精确度
plt.figure(figsize=(12, 5))

# 绘制损失
plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='训练损失')
plt.plot(history['val_loss'], label='验证损失')
plt.title('训练和验证损失')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 绘制精确度
plt.subplot(1, 2, 2)
plt.plot(history['accuracy'], label='训练精度')
plt.plot(history['val_accuracy'], label='验证精度')
plt.title('训练和验证精度')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'plot_training_history.npy'

In [4]:
# 绘制训练过程中的准确率和损失变化曲线
plt.figure(figsize=(12, 5))

# 绘制损失
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 绘制准确率
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

<IPython.core.display.Javascript object>

In [2]:
# 保存模型
model_save_path = 'D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/模型-0725/unet_3d_model_0726.h5'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

# 加载模型
model_load_path = 'D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/模型-0725/unet_3d_model_0726.h5'
loaded_model = tf.keras.models.load_model(model_load_path)
print(f"Model loaded from {model_load_path}")


Model saved to D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/模型-0725/unet_3d_model_0726.h5


Model loaded from D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/模型-0725/unet_3d_model_0726.h5


In [10]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# 加载模型
model = load_model('D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/模型-0725/unet_3d_model_0726.h5')

# 打印模型摘要
model.summary()

# 获取模型的所有层
layers = model.layers

# 打印每一层的名字和输出形状
for layer in layers:
    print(f'层名称: {layer.name}')
    print(f'输出形状: {layer.output_shape}')

# 打印模型的所有权重
for layer in layers:
    weights = layer.get_weights()
    print(f'层名称: {layer.name}')
    print(f'权重: {weights}')

# 打印模型的配置
config = model.get_config()
print('模型配置:')
print(config)

# 打印模型的优化器配置
optimizer_config = model.optimizer.get_config()
print('优化器配置:')
print(optimizer_config)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 32, 32, 64, 1)     │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d (Conv3D)               │ (None, 32, 32, 64, 64)    │           1,792 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_1 (Conv3D)             │ (None, 32, 32, 64, 64)    │         110,656 │ conv3d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d (MaxPooling3D)  │ (None, 16, 16, 32, 64)    │               0 │ conv3d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_2 (Conv3D)             │ (None, 16, 16, 32, 128)   │         221,312 │ max_pooling3d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_3 (Conv3D)             │ (None, 16, 16, 32, 128)   │         442,496 │ conv3d_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_1               │ (None, 8, 8, 16, 128)     │               0 │ conv3d_3[0][0]             │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_4 (Conv3D)             │ (None, 8, 8, 16, 256)     │         884,992 │ max_pooling3d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_5 (Conv3D)             │ (None, 8, 8, 16, 256)     │       1,769,728 │ conv3d_4[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_2               │ (None, 4, 4, 8, 256)      │               0 │ conv3d_5[0][0]             │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_6 (Conv3D)             │ (None, 4, 4, 8, 512)      │       3,539,456 │ max_pooling3d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_7 (Conv3D)             │ (None, 4, 4, 8, 512)      │       7,078,400 │ conv3d_6[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_transpose              │ (None, 8, 8, 16, 256)     │       1,048,832 │ conv3d_7[0][0]             │
│ (Conv3DTranspose)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 8, 8, 16, 512)     │               0 │ conv3d_transpose[0][0],    │
│                               │                           │                 │ conv3d_5[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_8 (Conv3D)             │ (None, 8, 8, 16, 256)     │       3,539,200 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 22,393,795 (85.43 MB)

 Trainable params: 22,393,793 (85.43 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

层名称: input_layer


AttributeError: 'InputLayer' object has no attribute 'output_shape'

In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# 加载模型
model = load_model('D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/模型-0725/unet_3d_model_0726.h5')

# 打印模型摘要
model.summary()

# 获取模型的所有层
layers = model.layers

# 打印每一层的名字和输出形状
for layer in layers:
    print(f'层名称: {layer.name}')
    # 检查是否有输出形状属性
    if hasattr(layer, 'output_shape'):
        print(f'输出形状: {layer.output_shape}')
    else:
        print('该层没有输出形状属性')

# 打印模型的所有权重
for layer in layers:
    weights = layer.get_weights()
    print(f'层名称: {layer.name}')
    if weights:
        print(f'权重: {weights}')
    else:
        print('该层没有权重')

# 打印模型的配置
config = model.get_config()
print('模型配置:')
print(config)

# 打印模型的优化器配置
if model.optimizer:
    optimizer_config = model.optimizer.get_config()
    print('优化器配置:')
    print(optimizer_config)
else:
    print('该模型没有优化器配置')


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 32, 32, 64, 1)     │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d (Conv3D)               │ (None, 32, 32, 64, 64)    │           1,792 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_1 (Conv3D)             │ (None, 32, 32, 64, 64)    │         110,656 │ conv3d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d (MaxPooling3D)  │ (None, 16, 16, 32, 64)    │               0 │ conv3d_1[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_2 (Conv3D)             │ (None, 16, 16, 32, 128)   │         221,312 │ max_pooling3d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_3 (Conv3D)             │ (None, 16, 16, 32, 128)   │         442,496 │ conv3d_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_1               │ (None, 8, 8, 16, 128)     │               0 │ conv3d_3[0][0]             │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_4 (Conv3D)             │ (None, 8, 8, 16, 256)     │         884,992 │ max_pooling3d_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_5 (Conv3D)             │ (None, 8, 8, 16, 256)     │       1,769,728 │ conv3d_4[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling3d_2               │ (None, 4, 4, 8, 256)      │               0 │ conv3d_5[0][0]             │
│ (MaxPooling3D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_6 (Conv3D)             │ (None, 4, 4, 8, 512)      │       3,539,456 │ max_pooling3d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_7 (Conv3D)             │ (None, 4, 4, 8, 512)      │       7,078,400 │ conv3d_6[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_transpose              │ (None, 8, 8, 16, 256)     │       1,048,832 │ conv3d_7[0][0]             │
│ (Conv3DTranspose)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 8, 8, 16, 512)     │               0 │ conv3d_transpose[0][0],    │
│                               │                           │                 │ conv3d_5[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv3d_8 (Conv3D)             │ (None, 8, 8, 16, 256)     │       3,539,200 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 22,393,795 (85.43 MB)

 Trainable params: 22,393,793 (85.43 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

层名称: input_layer
该层没有输出形状属性
层名称: conv3d
该层没有输出形状属性
层名称: conv3d_1
该层没有输出形状属性
层名称: max_pooling3d
该层没有输出形状属性
层名称: conv3d_2
该层没有输出形状属性
层名称: conv3d_3
该层没有输出形状属性
层名称: max_pooling3d_1
该层没有输出形状属性
层名称: conv3d_4
该层没有输出形状属性
层名称: conv3d_5
该层没有输出形状属性
层名称: max_pooling3d_2
该层没有输出形状属性
层名称: conv3d_6
该层没有输出形状属性
层名称: conv3d_7
该层没有输出形状属性
层名称: conv3d_transpose
该层没有输出形状属性
层名称: concatenate
该层没有输出形状属性
层名称: conv3d_8
该层没有输出形状属性
层名称: conv3d_9
该层没有输出形状属性
层名称: conv3d_transpose_1
该层没有输出形状属性
层名称: concatenate_1
该层没有输出形状属性
层名称: conv3d_10
该层没有输出形状属性
层名称: conv3d_11
该层没有输出形状属性
层名称: conv3d_transpose_2
该层没有输出形状属性
层名称: concatenate_2
该层没有输出形状属性
层名称: conv3d_12
该层没有输出形状属性
层名称: conv3d_13
该层没有输出形状属性
层名称: conv3d_14
该层没有输出形状属性
层名称: input_layer
该层没有权重
层名称: conv3d
权重: [array([[[[[ 0.03827731,  0.05120293,  0.0254327 , ..., -0.00385618,
           -0.00051847, -0.04119469]],

         [[ 0.05893854,  0.02691654,  0.04461911, ...,  0.04901291,
           -0.04515016, -0.02858291]],

         [[-0.01564573, -0.05766293, -0.05621489, ..

In [3]:
'''
模型用于预测，以及结果可视化
'''

# 开启交互旋转
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from scipy.spatial import KDTree
from collections import defaultdict
from itertools import combinations

# 加载obj文件
def load_obj_file(file_path):
    vertices = []
    faces = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = [float(parts[1]), float(parts[2]), float(parts[3])]
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                    faces.append(face)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return vertices, faces

# 将顶点平移至包围盒中心
def center_vertices(vertices):
    vertices_array = np.array(vertices)
    min_coords = vertices_array.min(axis=0)
    max_coords = vertices_array.max(axis=0)
    center = (min_coords + max_coords) / 2
    centered_vertices = vertices_array - center
    return centered_vertices.tolist()

# 点云转体素网格
def create_voxel_grid(data, grid_size):
    grid = np.zeros((grid_size, grid_size, grid_size))
    min_coords = np.min(data, axis=0)
    max_coords = np.max(data, axis=0)
    voxel_dim = (max_coords - min_coords) / grid_size

    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)  # Ensure indices are within bounds
        grid[voxel[0], voxel[1], voxel[2]] = 1

    return grid, min_coords, voxel_dim

# 旋转点云
def rotate_points(points, angles):
    x_angle, y_angle, z_angle = angles
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(x_angle), -np.sin(x_angle)],
        [0, np.sin(x_angle), np.cos(x_angle)]
    ])
    Ry = np.array([
        [np.cos(y_angle), 0, np.sin(y_angle)],
        [0, 1, 0],
        [-np.sin(y_angle), 0, np.cos(y_angle)]
    ])
    Rz = np.array([
        [np.cos(z_angle), -np.sin(z_angle), 0],
        [np.sin(z_angle), np.cos(z_angle), 0],
        [0, 0, 1]
    ])
    R = Rz @ Ry @ Rx
    return points @ R.T

# 从训练好的模型获取标签
def get_labels_from_model(model, voxel_grid):
    voxel_grid = np.expand_dims(voxel_grid, axis=0)  # Add batch dimension
    voxel_grid = np.expand_dims(voxel_grid, axis=-1)  # Add channel dimension
    predictions = model.predict(voxel_grid)
    labels = (predictions > 0.5).astype(int)
    return labels.reshape(voxel_grid.shape[1], voxel_grid.shape[2], voxel_grid.shape[3])

# 应用预测标签到原始点云
def apply_labels_to_point_cloud(data, predicted_labels, min_coords, voxel_dim, grid_size):
    labels = np.zeros(len(data))
    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)  # Ensure indices are within bounds
        labels[i] = predicted_labels[voxel[0], voxel[1], voxel[2]]
    return labels

# 识别并合并分割线段
def find_boundary_edges(vertices, faces, labels):
    edges = defaultdict(list)
    for face in faces:
        for (v1, v2) in combinations(face, 2):
            if labels[v1] != labels[v2]:
                edges[tuple(sorted([v1, v2]))].append(face)
    
    # Create an adjacency list for edges
    adjacency_list = defaultdict(list)
    for (v1, v2), faces in edges.items():
        adjacency_list[v1].append(v2)
        adjacency_list[v2].append(v1)
    
    # Find all connected components of edges
    visited = set()
    boundary_lines = []
    for vertex in adjacency_list:
        if vertex not in visited:
            stack = [vertex]
            boundary_line = []
            while stack:
                current = stack.pop()
                if current not in visited:
                    visited.add(current)
                    boundary_line.append(current)
                    for neighbor in adjacency_list[current]:
                        if neighbor not in visited:
                            stack.append(neighbor)
            boundary_lines.append(boundary_line)
    
    # Calculate the length of each boundary line and sort by length
    boundary_lines = sorted(boundary_lines, key=lambda line: sum(np.linalg.norm(vertices[line[i]] - vertices[line[i + 1]]) for i in range(len(line) - 1)), reverse=True)
    return boundary_lines

# 绘制带有分类标签的点云和分界线
def plot_surface_with_marks(vertices, faces, labels, part1, part2, view_angles=(30, 30), angles=(0, 0, 0)):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    vertices = np.array(vertices)
    faces = np.array(faces)

    # Apply rotation
    vertices = rotate_points(vertices, angles)
    
    x, y, z = vertices.T

    try:
        # 标记分割的点云部分
        part1_faces = [face for face in faces if sum(labels[vertex] == 0 for vertex in face) > 1]
        part2_faces = [face for face in faces if sum(labels[vertex] == 1 for vertex in face) > 1]
        
        part1_faces = np.array(part1_faces)
        part2_faces = np.array(part2_faces)
        
        if len(part1_faces) > 0:
            ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=part1_faces, color='cornflowerblue', alpha=0.6)
        if len(part2_faces) > 0:
            ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=part2_faces, color='honeydew', alpha=0.6)

        # 找到边界线并绘制最长的边界线
        boundary_lines = find_boundary_edges(vertices, faces, labels)
        if boundary_lines:
            longest_boundary_line = boundary_lines[0]
            ax.plot(vertices[longest_boundary_line, 0], vertices[longest_boundary_line, 1], vertices[longest_boundary_line, 2], color='red')
            
    except ValueError as e:
        print(f"ValueError: {e}")
        return

    # 设置标签和标题
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_title('3D Model with Segmentation and Boundary Lines')

    # 确保坐标轴刻度一致
    max_range = np.array([max(x)-min(x), max(y)-min(y), max(z)-min(z)]).max()
    mid_x = (max(x) + min(x)) * 0.5
    mid_y = (max(y) + min(y)) * 0.5
    mid_z = (max(z) + min(z)) * 0.5
    ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
    ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
    ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

    # 设置视角
    elev, azim = view_angles
    ax.view_init(elev=elev, azim=azim)  # Adjust these values as needed

    # 确保坐标轴比例相等
    ax.set_box_aspect([1,1,1])  # Aspect ratio is 1:1:1

    # 启用交互式旋转
    plt.show()

# 加载点云数据
obj_file_path = r'D:/BaiduSyncdisk/新建文件夹/桌面文件/深度学习程序/肩台外侧点_0715/150颗/60.obj'
vertices, faces = load_obj_file(obj_file_path)
centered_vertices = center_vertices(vertices)

# 将点云转换为体素网格
grid_size = 16  # Ensure grid size matches model requirements
voxel_grid, min_coords, voxel_dim = create_voxel_grid(np.array(centered_vertices), grid_size)

# 使用训练好的模型进行预测
predicted_labels = get_labels_from_model(model, voxel_grid)

# 获取原始点云的预测标签
predicted_point_labels = apply_labels_to_point_cloud(np.array(centered_vertices), predicted_labels, min_coords, voxel_dim, grid_size)

# 定义分割的部分（需要根据实际情况定义）
part1 = {i for i, label in enumerate(predicted_point_labels) if label == 0}
part2 = {i for i, label in enumerate(predicted_point_labels) if label == 1}

# 画图
plot_surface_with_marks(centered_vertices, faces, predicted_point_labels, part1, part2, view_angles=(30, 30), angles=(np.radians(90), np.radians(-30), np.radians(30)))


ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 32, 32, 64, 1), found shape=(1, 16, 16, 16)